## CODIGO


# Model Design

In [103]:
# Model design
#pip install agentpy
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import IPython

# Aux
import random

## Model Definition

In [104]:
class VehicleAgent(ap.Agent):

  """ An agent tha simulates a vehicle """

  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.xUbication = 0
    self.yUbication = 0
    self.fuel = 0
    self.status = "looking"
    self.stepsParking = 0
    self.parkingCoordinates = None

  def moveUp(self):
    self.map.move_by(self, (-1, 0))

  def park(self):
    self.status = 'parked'

  def findParking(self):

    # Obtenemos estacionamientos libres en el grid
    pAgents = [i for i in self.model.map.agents if 'ParkingAgent' in i.type and i.availability]
    
    if len(pAgents) > 0:
      minValue = float('inf')
      for parkA in pAgents:
        current = self.map.neighbors(self, distance = 0)
        if current.direction == 'North' and parkA.north != None:
          if parkA.north < minValue:
            minValue = parkA.north
            self.parkingCoordinates = self.map.positions[parkA]
        elif current.direction == 'South' and parkA.south != None:
          if parkA.south < minValue:
            minValue = parkA.south
            self.parkingCoordinates = self.map.positions[parkA]
        elif current.direction == 'East' and parkA.east != None:
          if parkA.east < minValue:
            minValue = parkA.east
            self.parkingCoordinates = self.map.positions[parkA]
        elif current.direction == 'West' and parkA.west != None:
          if parkA.west < minValue:
            minValue = parkA.west
            self.parkingCoordinates = self.map.positions[parkA]

      #print(current.type)

      #print(parkA.type)

  def movement(self):
    # Obtenemos los vecinos al vehiculo
    neighbors = self.map.neighbors(self, distance = 0)
    roadNeighbors = [i for i in neighbors if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]

    # Observamos los posibles movimientos <- Aqui logica de mejor mov al estacionamiento
    # Movimiento cuando la celda va en un sólo sentido o cuando el carro está dejando el mapa.
    if len(roadNeighbors) == 1 or self.parkingCoordinates == None:
      # Camino hacia la arriba
      if(roadNeighbors[0].direction == 'North'):
        self.map.move_by(self, (-1, 0))

      elif(roadNeighbors[0].direction == 'South'):
        self.map.move_by(self, (1, 0))

      elif(roadNeighbors[0].direction == 'West'):
        self.map.move_by(self, (0, -1))
      
      else:
        self.map.move_by(self, (0, 1))
    elif len(roadNeighbors) > 1 and self.parkingCoordinates != None:
      # Lógica para el cruce
      decision = random.randint(0, 1)
      if(roadNeighbors[decision].direction == 'North'):
        self.map.move_by(self, (-1, 0))

      elif(roadNeighbors[decision].direction == 'South'):
        self.map.move_by(self, (1, 0))

      elif(roadNeighbors[decision].direction == 'West'):
        self.map.move_by(self, (0, -1))
      
      else:
        self.map.move_by(self, (0, 1))
    else:
      pass
      
    
    # Generamos el movimiento


In [105]:
class ParkingAgent(ap.Agent):

  """ An agent that simulates a parking slot """

  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.id
    self.availability = True
    self.xUbication = 0
    self.yUbication = 0
    self.north = None
    self.south = None
    self.east = None
    self.west = None
    
  def setAvailability(self):
    # Condicional en dependencia a los agentes vehiculo
    pass


      


In [106]:
def randomPos(n):
    positions = [(0,4), (11,0), (15, 11), (4, 15)]
    return random.sample(positions, n)


In [107]:
class CongestionModel(ap.Model):

  """ A simple model of random wealth transfers """

  def setup(self):

    # Creacion de Grid
    self.map = ap.Grid(self, (16, 16), track_empty = True)

    # Creacion de Agentes
    self.vehicleAgents = []
    self.parkingAgents = ap.AgentList(self, 10, ParkingAgent)
    distanceMatrix = [[22,8,29,15],
                      [15,29,22,8],
                      [None,None,None,2],
                      [21,7,28,14],
                      [10,24,17,31],
                      [23,9,30,16],
                      [31,17,10,24],
                      [8,22,15,29],
                      [None,None,3,None],
                      [29,15,8,22]]
    for i in range(0, 10, 1): 
      self.parkingAgents[i].north = distanceMatrix[i][0]
      self.parkingAgents[i].south = distanceMatrix[i][1]
      self.parkingAgents[i].east = distanceMatrix[i][2]
      self.parkingAgents[i].west = distanceMatrix[i][3]

    parkPositions = [(3,1), (3,8), (3,14), (6,5), (6,10), (8,3), (10,9), (10,14), (12,2), (12,7)]
    

    # Agente camino
    # Camino hacia arriba
    road = []
    pos = []

    for i in range(0,4,1):
      
      temp = ap.AgentList(self, 16)
      
      if (i == 0):
        temp.direction = "East"
        for j in range(0,16,1):
          pos += [(11,j)]
      elif (i == 1):
        temp.direction = "West"
        for j in range(0,16,1):
          pos += [(4,j)]
      elif (i == 2):
        temp.direction = "South"
        for j in range(0,16,1):
          pos += [(j,4)]
      else:
        temp.direction = "North"
        for j in range(0,16,1):
          pos += [(j,11)]
      
      road += temp
    
    self.roads = road
    

    #self.map.add_agents(self.vehicleAgents, random=True, empty=True)
    
    self.map.add_agents(self.parkingAgents, positions = parkPositions)
    self.map.add_agents(self.roads, positions = pos)

    #self.map.move_by(self.parkingAgents.random(), (3,3))

    #self.vehicleAgents[0].findParking()

    """ self.map.move_by(self.vehicleAgents.random(), (3,3))
    self.map.move_by(self.parkingAgents.random(), (3,3)) """
 
  def update(self):
    if self.t % self.p.spawnIntensity == 0:

      #self.record('Traffic', traffic())
      numCars = random.randint(0,3)
      positions = randomPos(numCars)

      newCars = ap.AgentList(self, numCars, VehicleAgent)

      self.vehicleAgents += newCars

      if(len(positions) > 0):
        self.map.add_agents(newCars, positions = (positions))
        newCars.findParking()

  def step(self):
    vehAgents = [i for i in self.map.agents if 'VehicleAgent' in i.type]

    for vehicle in vehAgents:
      vehicle.movement()

    #print(positions)


  def end(self):
    pass
    #self.agents.record('')

## Simulation Run

In [108]:
parameters = {
  #'vehicleAgents': 10,
  #'parkingAgents': 3,
  'spawnIntensity': 5,
  'steps': 100,
  'seed': 42,
}

model = CongestionModel(parameters)
#results = model.run()
#results

In [109]:
# Create single-run animation with custom colors

def view_agents(Agent):
  num = 0
  
  if('VehicleAgent' in Agent.type):
    num = 0
  elif('ParkingAgent' in Agent.type):
    num = 1
  elif('Agent' in Agent.type):
    num = 2
  else:
    num = 3

  return num

def animation_plot(model, ax):
    attr_grid = model.map.apply(view_agents)
    #print(attr_grid)
    #attr_grid = model.map.attr_grid(isinstance())
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#848484', 3:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    #ax.set_title(f"Simulation of a forest fire\n"
                 #f"Time-step: {model.t}, Trees left: "
                 #f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = CongestionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=10))